# Imports

In [47]:
import json
import math
import re
import time
from datetime import datetime
from string import punctuation, whitespace

import emoji
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from natasha import (PER, Doc, MorphVocab, NamesExtractor, NewsEmbedding,
                     NewsMorphTagger, NewsNERTagger, NewsSyntaxParser,
                     Segmenter)
from nltk.corpus import stopwords
from selenium import webdriver

In [48]:
stopwords_ru = stopwords.words("russian")
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

# Parsing

In [49]:
with open("./data/last_parsed_html.txt", "r+", encoding="UTF8") as f:
    html = "".join(f.readlines())
    soup = BeautifulSoup(html)

In [50]:
reviews = soup.findAll("li", class_="comments__item feedback j-feedback-slide")

In [51]:
res = []
for review in reviews:
    text = review.find("p", class_="feedback__text")
    title_element = review.find("span", class_=lambda value: value and value.startswith("feedback__rating stars-line"))
    res.append({"full_text": text.text, "rating": title_element["class"][-1][-1]})
soup.decompose()

In [52]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 10)
df = pd.DataFrame.from_dict(res)
df.head(10)

,full_text,rating
0,Отличный горшок. Пришло всё целое. Спасибо за ...,5
1,Качественный горшок,5
2,"Удобные стильные горшки , покупаю не первый раз .",5
3,Классный горшок для цветов заказываю уже трети...,5
4,"Спасибо,горшки красивые,но немного обидно,что ...",5
5,"Все замечательно, хорошо упакован и очень прия...",5
6,"Очень довольна горшками,индикатор работает отл...",5
7,"Горшок большой, все пришло целое!",5
8,Описание крайне не верное. Горшок напольный. О...,2
9,👍👍👍👍👍,5


In [53]:
df.describe()

,full_text,rating
count,698,698
unique,687,5
top,Хороший горшок,5
freq,5,547


In [54]:
df

,full_text,rating
0,Отличный горшок. Пришло всё целое. Спасибо за ...,5
1,Качественный горшок,5
2,"Удобные стильные горшки , покупаю не первый раз .",5
3,Классный горшок для цветов заказываю уже трети...,5
4,"Спасибо,горшки красивые,но немного обидно,что ...",5
...,...,...
693,Доброго дня! Вместо горшка на 9л. прислали на ...,5
694,Отличный горшок. Удобная система полива. Выгля...,5
695,"Отличный горшок, теперь хочу собрать всю колле...",5
696,"Хороший горшок, нужного мне размера. Правильна...",5


# Preprocessing

## Emoji

In [55]:
remove_list = list(emoji.EMOJI_DATA.keys()) + list("0123456789")

df["full_text"] = df["full_text"].map(
    lambda x: "".join(c for c in x if c not in remove_list)
)
# extract_emojis("👌💚все понравилось , закажу еще один")
df = df.drop(df[df["full_text"].map(lambda x: x == "")].index, axis=0)
df.head(10)

,full_text,rating
0,Отличный горшок. Пришло всё целое. Спасибо за ...,5
1,Качественный горшок,5
2,"Удобные стильные горшки , покупаю не первый раз .",5
3,Классный горшок для цветов заказываю уже трети...,5
4,"Спасибо,горшки красивые,но немного обидно,что ...",5
5,"Все замечательно, хорошо упакован и очень прия...",5
6,"Очень довольна горшками,индикатор работает отл...",5
7,"Горшок большой, все пришло целое!",5
8,Описание крайне не верное. Горшок напольный. О...,2
10,Очень интересный горшочек. Удобная колба для к...,5


# Speller

In [56]:
df['full_text_Count'] = df['full_text'].str.len()
df['full_text_Count'].describe()

count    696.000000
mean     112.466954
std      118.909150
min       10.000000
25%       38.750000
50%       78.000000
75%      140.250000
max      980.000000
Name: full_text_Count, dtype: float64

In [57]:
df.iloc[3]["full_text"]
df[df['full_text_Count'] >= df['full_text_Count'].describe()["75%"]]

,full_text,rating,full_text_Count
3,Классный горшок для цветов заказываю уже трети...,5,220
8,Описание крайне не верное. Горшок напольный. О...,2,191
10,Очень интересный горшочек. Удобная колба для к...,5,244
11,Горшок очень симпатичный. На вид горшок большо...,5,377
12,Отличное кашпо!!! Всё в комплекте!!! Идеально ...,5,271
...,...,...,...
678,"Во-первых горшок состоит из двух, когда брал д...",3,296
686,Горшочек очень понравился!!! К сезону отпусков...,5,171
691,"Из трех горшков объёмом литров,один был с диф...",5,220
695,"Отличный горшок, теперь хочу собрать всю колле...",5,149


In [58]:
def speller_api(texts, df, first_index, last_index):
    payload = {'text': texts, 'options': 526} 
    try: 
        r = requests.post('https://speller.yandex.net/services/spellservice.json/checkTexts?', data=payload)
        r.encoding = 'utf-8'
        res = r.json()
        print(res)
        for i in range(len(texts)):
            df_i = i + first_index
            # Добавить число с которого начинается отсчет
            for mistake_ind in range(len(res[i])):
                suggestion = res[i][mistake_ind]
                if suggestion["s"]:
                    print(df.iloc[df_i]["full_text"])
                    df.iloc[df_i]["full_text"] = (
                        df["full_text"].iloc[df_i][: suggestion["pos"]]
                        + suggestion["s"][0]
                        + df["full_text"].iloc[df_i][suggestion["pos"] + suggestion["len"] :]
                    )
                    print(df.iloc[df_i]["full_text"])
    except Exception as e:
        print(e)
        print(r.text)

In [59]:
first_index = 0
max_sum = 10000
last_index = 0
for last_index, row in df.iterrows():
    if df.iloc[first_index:last_index]["full_text_Count"].sum() >= max_sum:
        texts = df.iloc[first_index:last_index]["full_text"]
        # print(df.iloc[first_index:last_index]["full_text_Count"].sum(), len(texts))
        speller_api(texts, df, first_index, last_index)
        first_index = last_index
texts = df.iloc[first_index:last_index + 1]["full_text"]
speller_api(texts, df, first_index, last_index)

[[], [], [], [], [], [{'code': 1, 'pos': 58, 'row': 0, 'col': 58, 'len': 11, 'word': 'обнаружил а', 's': ['обнаружила']}], [], [], [{'code': 1, 'pos': 16, 'row': 0, 'col': 16, 'len': 9, 'word': 'не верное', 's': ['неверное', 'не верное']}], [], [{'code': 1, 'pos': 71, 'row': 0, 'col': 71, 'len': 10, 'word': 'не большой', 's': ['небольшой']}, {'code': 1, 'pos': 199, 'row': 0, 'col': 199, 'len': 6, 'word': 'нечего', 's': ['ничего']}], [{'code': 1, 'pos': 140, 'row': 0, 'col': 140, 'len': 5, 'word': 'балов', 's': ['баллов']}, {'code': 1, 'pos': 236, 'row': 0, 'col': 236, 'len': 7, 'word': 'Вообщем', 's': ['в общем']}], [], [], [], [], [], [], [], [], [{'code': 1, 'pos': 120, 'row': 0, 'col': 120, 'len': 10, 'word': 'резерваура', 's': ['резервуара']}, {'code': 1, 'pos': 178, 'row': 0, 'col': 178, 'len': 10, 'word': 'на сколько', 's': ['насколько', 'на сколько']}], [], [], [], [], [], [], [], [], [], [{'code': 1, 'pos': 67, 'row': 0, 'col': 67, 'len': 7, 'word': 'пожесче', 's': ['пожестче']

C:\Users\Kirill\AppData\Local\Temp\ipykernel_4856\1259591670.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[df_i]["full_text"] = (


[[{'code': 1, 'pos': 77, 'row': 1, 'col': 7, 'len': 10, 'word': 'упакованый', 's': ['упакованный']}], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [{'code': 1, 'pos': 13, 'row': 0, 'col': 13, 'len': 10, 'word': 'не большой', 's': ['небольшой', 'не большой']}], [{'code': 1, 'pos': 166, 'row': 0, 'col': 166, 'len': 9, 'word': 'Преобрела', 's': ['Приобрела']}], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [{'code': 1, 'pos': 232, 'row': 0, 'col': 232, 'len': 12, 'word': 'самополивный', 's': ['самопаливный']}, {'code': 1, 'pos': 245, 'row': 0, 'col': 245, 'len': 7, 'word': 'горочек', 's': ['горшочек']}], [], [], [], [], [], [], [], [], [], [], [], [], [{'code': 1, 'pos': 296, 'row': 0, 'col': 296, 'len': 6, 'word': 'недели', 's': ['неделю', 'недели']}], [{'code': 1, 'pos': 122, 'row': 0, 'col': 122, 'len': 5, 'word': 'воцап', 's': ['вацап']}], [], [], [], [], [], [], [], [], [], [], [{'code': 1, 'pos': 151, 'row': 0, 'col': 

In [60]:
payload = {'text': df["full_text"], 'options': 526} 
try: 
    # Максимум 10000 символов за раз. Обработки для этого нет, её, по хорошему, надо сделать
    r = requests.post('https://speller.yandex.net/services/spellservice.json/checkTexts?', data=payload)
    r.encoding = 'utf-8'
    res = r.json()
    print(res)
except Exception as e:
    print(e)
    print(r.text)

[[], [], [], [], [], [{'code': 1, 'pos': 58, 'row': 0, 'col': 58, 'len': 11, 'word': 'обнаружил а', 's': ['обнаружила']}], [], [], [{'code': 1, 'pos': 16, 'row': 0, 'col': 16, 'len': 9, 'word': 'не верное', 's': ['неверное', 'не верное']}], [], [{'code': 1, 'pos': 71, 'row': 0, 'col': 71, 'len': 10, 'word': 'не большой', 's': ['небольшой']}, {'code': 1, 'pos': 199, 'row': 0, 'col': 199, 'len': 6, 'word': 'нечего', 's': ['ничего']}], [{'code': 1, 'pos': 140, 'row': 0, 'col': 140, 'len': 5, 'word': 'балов', 's': ['баллов']}, {'code': 1, 'pos': 236, 'row': 0, 'col': 236, 'len': 7, 'word': 'Вообщем', 's': ['в общем']}], [], [], [], [], [], [], [], [], [{'code': 1, 'pos': 120, 'row': 0, 'col': 120, 'len': 10, 'word': 'резерваура', 's': ['резервуара']}, {'code': 1, 'pos': 178, 'row': 0, 'col': 178, 'len': 10, 'word': 'на сколько', 's': ['насколько', 'на сколько']}], [], [], [], [], [], [], [], [], [], [{'code': 1, 'pos': 67, 'row': 0, 'col': 67, 'len': 7, 'word': 'пожесче', 's': ['пожестче']

In [ ]:
# Не копируется датафрейм, точнее не вставляются значения в "копию"
# df.iloc[0, "full_text"] = "123"
for i in range(len(df["full_text"])):
    for mistake_ind in range(len(res[i])):
        suggestion = res[i][mistake_ind]
        if suggestion["s"]:
            df.iloc[i, 'full_text'] = (
                df["full_text"].iloc[i][: suggestion["pos"]]
                + suggestion["s"][0]
                + df["full_text"].iloc[i][suggestion["pos"] + suggestion["len"] :]
            )

df


In [121]:
df.iloc[50]

full_text          Ребят, вы так и пишите, что горшок  л, а не . ...
rating                                                             1
full_text_Count                                                  108
Name: 51, dtype: object

# Natasha

In [123]:
def get_syntax(df, row, text_cell = "text"):
    res = []
    
    def recursive(df, row, res, text_cell):
        edges = df[df["head_id"]==row["id"]]
        if edges.empty:
            if re.search(fr"[{punctuation}]", row[text_cell]):
                return
            res.append(row[text_cell])
            return
        
        isPrinted = False
        for index, edge in edges.iterrows():
            if edge["text"] == "(":
                k = 0
            if edge["rel"] == "conj":
                continue
            if text_cell == "lemma":
                if edge["text"] in stopwords_ru or re.search(fr"\d|[{punctuation}]", edge["text"]):
                    continue
            if int(edge["id"].split("_")[1]) > int(row["id"].split("_")[1]) and not isPrinted:
                res.append(row[text_cell])
                isPrinted = True
            recursive(df, edge, res, text_cell)
        if not isPrinted:
            res.append(row[text_cell])
            isPrinted = True
    
    recursive(df,row, res, text_cell)
    if text_cell == "text":
        res = re.sub(r'\s([?.!,;:"](?:\s|$))', r'\1', " ".join(res))
    return res

In [124]:
pd.set_option('display.max_rows', 10)

In [125]:
resarr = []
# try:
for df_item in df.to_numpy():
    res = df_item[df.columns.get_indexer(["text"])[0]]
    # res = df_item[0]
    doc = Doc(res)  # Doc(res.text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    data = doc.tokens
    columns = list(doc.tokens[0].as_json.keys())
    df_natasha = pd.DataFrame(data=data, columns=columns)

    df_natasha2 = df_natasha[
        (df_natasha["rel"] == "root") | (df_natasha["rel"] == "conj")
    ]
    # print(df_natasha)
    # print(df_natasha2)
    if df_natasha2.empty:
        df_natasha2 = df_natasha[df_natasha["id"] == df_natasha["head_id"]]
        if df_natasha2.empty:
            # print(df_natasha)
            # Если наташа не смогла определить подлежащее, то обычно отзыв бессмысленен
            continue
        df_natasha.loc[df_natasha2.index, ["head_id"]] = df_natasha2["head_id"].replace(
            to_replace=r"^(\d+_)(\d+)$", value=r"\g<1>" + "0", regex=True
        )
        df_natasha2 = df_natasha.loc[df_natasha2.index]
        # print(df_natasha2)
        df_natasha["syntax"] = df_natasha2.apply(
            lambda row: get_syntax(df_natasha, row), axis=1
        )
        df_natasha["syntax_lemmas"] = df_natasha2.apply(
            lambda row: get_syntax(df_natasha, row, text_cell="lemma"), axis=1
        )
        # print(df_natasha)
        df_natasha2 = df_natasha.loc[df_natasha2.index]
    else:
        df_natasha["syntax"] = df_natasha2.apply(
            lambda row: get_syntax(df_natasha, row), axis=1
        )
        df_natasha["syntax_lemmas"] = df_natasha2.apply(
            lambda row: get_syntax(df_natasha, row, text_cell="lemma"), axis=1
        )
        df_natasha2 = df_natasha[
            (df_natasha["rel"] == "root") | (df_natasha["rel"] == "conj")
        ]

    texts = df_natasha2["syntax"].values
    lemmas = df_natasha2["syntax_lemmas"].values
    for i in range(len(texts)):
        if len(texts[i]) < 2 or len(lemmas[i]) == 0:
            continue
        resarr.append(
            {
                "text": texts[i][0].upper() + texts[i][1:],
                "full_text": res,
                "class": 0,
                "lemmas": lemmas[i],
                "rating": df_item[1],
            }
        )
# except Exception as e:
#     print(e)
df_res = pd.DataFrame(resarr)
df_res


,text,full_text,class,lemmas,rating
0,Отличный горшок,Отличный горшок. Пришло всё целое. Спасибо за ...,0,"[отличный, горшок]",5
1,Пришло всё целое,Отличный горшок. Пришло всё целое. Спасибо за ...,0,"[прийти, весь, целый]",5
2,Спасибо за подарочек-удобрение,Отличный горшок. Пришло всё целое. Спасибо за ...,0,[спасибо],5
3,Качественный горшок,Качественный горшок,0,"[качественный, горшок]",5
4,Удобные стильные горшки покупаю не первый раз,"Удобные стильные горшки , покупаю не первый раз .",0,"[удобный, стильный, горшок, покупать]",5
...,...,...,...,...,...
2602,Но с автополивом заказала первый раз а,"Очень понравился горшок, но с автополивом зака...",0,"[автополив, заказать]",5
2603,Поэтому не учла литров что внутренний на где т...,"Очень понравился горшок, но с автополивом зака...",0,"[поэтому, учесть, литр, внутренний, это, мален...",5
2604,Это объём внешнего горшка,"Очень понравился горшок, но с автополивом зака...",0,"[это, объем, внешний, горшок]",5
2605,Литра,"Очень понравился горшок, но с автополивом зака...",0,[литр],5


# Save

In [126]:
dt = datetime.utcnow().strftime('%Y-%m-%d-%H-%M-%S')
file_path = f"./data/unprepared_{dt}.xlsx"
df_res.to_excel(file_path, encoding="UTF-8")